In [1]:
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
import time

In [2]:

driver = webdriver.Chrome(executable_path='../chromedriver/chromedriver.exe')
driver.get('https://www.hltv.org/ranking/teams/')

C:\Users\Carlos\anaconda3\envs\csgo\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [4]:
years = driver.find_elements(By.CLASS_NAME,'filter-column-content  ')[0].text.split('\n')

full_data = []
df = pd.DataFrame(columns=['Date', 'Position', 'Teams', 'Players'])
for year in years:
    driver.find_element(By.PARTIAL_LINK_TEXT, year).click()
    months = driver.find_elements(By.CLASS_NAME,'filter-column-content  ')[1].text.split('\n')
    months_scrapped = []
    for month in months:
        if month in months and month not in months_scrapped:
            print('Scrapping {}-{}'.format(year, month))
            months_scrapped.append(month)
            driver.find_element(By.PARTIAL_LINK_TEXT, month).click()
            time.sleep(2)
            driver.find_element(By.CLASS_NAME, 'ranking-header').click()
            ranking = driver.find_element(By.CLASS_NAME,'ranking').text.split('\n')
            ranking_date = [ranking[1]]
            positions = []
            teams = []
            players = []

            for i in ranking:
                if i.startswith('#'):
                    item_index = ranking.index(i)
                    positions.append(i)
                    teams.append(ranking[item_index+1])
                    players.append(ranking[item_index+2])
            month_df = pd.DataFrame([ranking_date * len(teams), positions, teams, players]).T
            month_df.columns = ['Date', 'Position', 'Teams', 'Players']
        df = pd.concat([df, month_df])

df.to_csv('../data/ranking_raw.csv', index=False)


Scrapping 2021-November
Scrapping 2021-October
Scrapping 2021-September
Scrapping 2021-August
Scrapping 2021-July
Scrapping 2021-June
Scrapping 2021-May
Scrapping 2021-April
Scrapping 2021-March
Scrapping 2021-February
Scrapping 2021-January
Scrapping 2020-December
Scrapping 2020-November
Scrapping 2020-October
Scrapping 2020-September
Scrapping 2020-August
Scrapping 2020-July
Scrapping 2020-June
Scrapping 2020-May
Scrapping 2020-April
Scrapping 2020-March
Scrapping 2020-February
Scrapping 2020-January
Scrapping 2019-December
Scrapping 2019-November
Scrapping 2019-October
Scrapping 2019-September
Scrapping 2019-August
Scrapping 2019-July
Scrapping 2019-June
Scrapping 2019-May
Scrapping 2019-April
Scrapping 2019-March
Scrapping 2019-February
Scrapping 2019-January
Scrapping 2018-December
Scrapping 2018-November
Scrapping 2018-October
Scrapping 2018-September
Scrapping 2018-August
Scrapping 2018-July
Scrapping 2018-June
Scrapping 2018-May
Scrapping 2018-April
Scrapping 2018-March
Scrappi

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F606F3+2492147]
	Ordinal0 [0x00EF9BD1+2071505]
	Ordinal0 [0x00E02478+1057912]
	Ordinal0 [0x00E04BA4+1067940]
	Ordinal0 [0x00E04A6E+1067630]
	Ordinal0 [0x00E04CD0+1068240]
	Ordinal0 [0x00E2E0FC+1237244]
	Ordinal0 [0x00E23573+1193331]
	Ordinal0 [0x00E457C3+1333187]
	Ordinal0 [0x00E23466+1193062]
	Ordinal0 [0x00E458AA+1333418]
	Ordinal0 [0x00E54F38+1396536]
	Ordinal0 [0x00E4568B+1332875]
	Ordinal0 [0x00E221D4+1188308]
	Ordinal0 [0x00E2302F+1191983]
	GetHandleVerifier [0x010E67A6+1545030]
	GetHandleVerifier [0x0119105C+2243580]
	GetHandleVerifier [0x00FEBC97+518199]
	GetHandleVerifier [0x00FEAD80+514336]
	Ordinal0 [0x00EFED2D+2092333]
	Ordinal0 [0x00F02EE8+2109160]
	Ordinal0 [0x00F03022+2109474]
	Ordinal0 [0x00F0CB71+2149233]
	BaseThreadInitThunk [0x7565FA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F77A

In [16]:
pd.DataFrame(df['Teams'].str.split('(').str[0].unique().tolist()).to_csv('../data/teste.csv')

In [22]:
teams_id.columns

Index(['Unnamed: 0', '0', '1'], dtype='object')

In [24]:
teams_id = pd.read_csv('../data/teste.csv')
teams_id.dropna(axis=0, inplace=True)
teams_id['1'] = teams_id['1'].astype(int).astype(str)

In [27]:
url_matches = 'https://www.hltv.org/results?'
ids = teams_id['1'].tolist()
for i in ids:
    url_matches = url_matches + 'team={}&'.format(i)

driver.get(url_matches)